<a href="https://colab.research.google.com/github/estrickler1/RedShift/blob/main/nz40_Redshift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Galaxies' Redshift Catalog**

The author is Ethan Strickler

Mentor is Dr. Nicholas Jacob

Idea came from https://www.cfa.harvard.edu/~dfabricant/huchra/zcat/zcom.htm

![RedShiftExample](https://drive.google.com/uc?export=view&id=12KXKzdGP4ieRVxB6JJigNrFOt50neUzI)

In [90]:
import pandas as pa
import plotly.express as px
import math
import numpy as np
from scipy import stats

url = 'https://raw.githubusercontent.com/nurfnick/Galaxies/main/nz40.csv'
data = pa.read_csv(url)

url2 = 'https://raw.githubusercontent.com/estrickler1/RedShift/main/CFA2.csv'
data2 = pa.read_csv(url2)

url3 = 'https://raw.githubusercontent.com/estrickler1/RedShift/main/zwicky.csv'
zwicky = pa.read_csv(url3)

In [92]:
data = data[1:-7] #removing 
data2 = data2[1:-10]

In [93]:
data = data.iloc[:,0:8]
data2 = data2.iloc[:,0:8]

In [94]:
data

,Name,RA (1950),Dec m_zw,v_h,sig sour,Type,D1,D1.1
2,00006-0211,36,-2110014.3,7323,3112700,4X,1.3,0.8
3,N7814,41.1,15520312,1050,410611,2A s,6.5,2.7
4,00010+2256,100,22560014,7301,3412700,5,1.9,1
5,N7816,112,7110014,5241,510620,4,2,2
6,N7817,124.9,20281812.7,2309,512812,4A,4,1.1
...,...,...,...,...,...,...,...,...
2384,23541+1330,235406,13300014.3,10866,3112700,5,0.9,0.5
2385,23542+1633,235412,16330014.5,1788,3612700,10,2.6,0.9
2386,N7794,235600,10260013.8,5264,3112700,6 R,1.4,1.3
2387,N7798,235652.1,20283312.7,2407,810611,5B,1.4,1.3


This is the galaxy N7814 listed in row 3.

![N7814](https://drive.google.com/uc?export=view&id=1UTecn2UdNCUwyvEsOhz8VOHG21_YHr6b)

This is the galaxy N7806 listed in row 2392.

![N7806](https://drive.google.com/uc?export=view&id=1pRUJsWgqbbQBMKtDzP-Xk4IBMDNxKlZV)

Combine the different datasets into one table.

In [95]:
diction ={}
for i in range(8):
  diction[data2.columns[i]] = data.columns[i]
diction
data2 = data2.rename(columns = diction)

In [96]:
frames = [data,data2]

data = pa.concat(frames, ignore_index= True)


In [97]:
zwicky.columns

Index(['ra2000  ', 'Dec-00', 'Zmag', 'cz   ', 'cze', 'T', 'U', 'Ne',
       'Zname       ', 'C', 'Ref ', 'Oname      ', 'M', 'N'],
      dtype='object')

In [98]:
data.columns

Index(['Name', 'RA (1950)', 'Dec m_zw', 'v_h', 'sig sour', 'Type', 'D1',
       'D1.1'],
      dtype='object')

In [99]:
diction2 ={'ra2000  ':'RA (1950)', 'Dec-00':'Dec m_zw', 'cz   ':'v_h', 'Oname      ':'Name'}

zwicky = zwicky.rename(columns = diction2)
zwicky.shape

(19367, 14)

In [100]:
frames = [data, zwicky]

data = pa.concat(frames, ignore_index= True)

In [101]:
data[data['v_h'] == '1 5261']

,Name,RA (1950),Dec m_zw,v_h,sig sour,Type,D1,D1.1,Zmag,cze,T,U,Ne,Zname,C,Ref,M,N
63,N 182,3536,2270013.5,1 5261,600620,1X T,2.3,1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2749,N 182,3538.2,2271513.5,1 5261,600620,1X T,2.3,1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
data = data[~data.v_h.isnull()]


Deleting every row with a flawed $v_h$ value. Then putting the values that have not been deleted back into the table.


In [103]:
badnumbers = data.v_h.loc[data['v_h'].str.match(r'[0-9.]+ ')] #starts with a number and then a space
goodnumbers = badnumbers.str.slice(2,) #get rid of that # and space

In [104]:
 greatnumbers = data.v_h.loc[~data['v_h'].str.match(r'[0-9.]+ ')] #get all the ones that didn't have the above issue

In [105]:
somenumbers = pa.concat([greatnumbers,goodnumbers]) #put them back together

In [106]:
data['v_h'] = somenumbers

In [109]:
data = data.loc[~data['v_h'].str.contains(r'[A-z]')] #removing the ones that had months...
data = data.loc[~data['v_h'].str.contains(r' ')]

In [110]:
data.v_h.astype(int)

0         7323
1         1050
2         7301
3         5241
4         2309
         ...  
39942    19201
39943     5512
39944     8590
39945     7578
39946     7649
Name: v_h, Length: 39462, dtype: int64

In [111]:
data['v_h'] = pa.to_numeric(data['v_h'])

In [112]:
data.describe()

,v_h,Zmag,U,Ne
count,39462.000000,19204.000000,19204.000000,19204.000000
mean,7182.022781,14.912602,0.551760,0.145438
std,10666.863893,0.762431,0.855416,0.445358
min,-571.000000,4.300000,0.000000,0.000000
25%,4559.000000,14.700000,0.000000,0.000000
50%,6821.000000,15.200000,0.000000,0.000000
75%,9280.000000,15.400000,1.000000,0.000000
max,926681.000000,15.700000,4.000000,5.000000


In [122]:
data = data.astype({'Dec m_zw':str})

In [123]:
#data = data.loc[~data['Dec m_zw'].str.match(r'[0-9.]+ ')]
data = data.loc[~data['Dec m_zw'].str.contains(r'[A-z]')]
data = data.loc[~data['Dec m_zw'].str.contains(r' ')]
data = data[~(data['Dec m_zw'] == '-001331 9.7')]
decData = data["Dec m_zw"]

Right ascension is the angular distance of a particular point measured eastward along the celestial equator from the Sun at the March equinox to the point in question above the earth.

Take the given RA (1950) value, which is listed in hours, minutes, and seconds (HHMMSS), then convert it into degrees. One day is equal to 360 degrees.

In [124]:
raData = data["RA (1950)"]

In [125]:
raData = raData.astype(float)
raInDegrees = []

for point in raData:
  x = math.floor(point/10000)
  y = math.floor((point - x*10000)/100)
  z = point - x*10000-y*100
  decimal = x + y/60 + z/3600
  degrees = decimal *360/24
  raInDegrees.append(degrees)

raInDegrees

data["RADegrees"] = raInDegrees 


Declination's angle is measured north or south of the celestial equator.

Take the given Dec m_zw data, which is given in degrees, minutes, seconds, and milliseconds (DDDMMSSmm), then convert it to degrees.

In [126]:
data["Dec m_zw"]

0        -2110014.3
1          15520312
2          22560014
3           7110014
4        20281812.7
            ...    
39942         25627
39943        125818
39944         40528
39945        271238
39946         32107
Name: Dec m_zw, Length: 39446, dtype: object

In [127]:
decData = decData.astype(float)

decInDegrees = []

for point in decData:
  if point>0:
    w = math.floor(point/1000000)
    x = math.floor((point - w*1000000)/10000)
    y = math.floor((point - w*1000000-x*10000)/100)
    z = point - w*1000000 - x*10000 - y*100
    decimal = x/60 + y/3600 + z/(3.6*10**6)
    degrees = decimal *360/24 + w
    decInDegrees.append(degrees)
  else:
    point = abs(point)
    w = math.floor(point/1000000)
    x = math.floor((point - w*1000000)/10000)
    y = math.floor((point - w*1000000-x*10000)/100)
    z = point - w*1000000 - x*10000 - y*100
    decimal = x/60 + y/3600 + z/(3.6*10**6)
    degrees = decimal *360/24 + w
    decInDegrees.append(-degrees)

data["DecDegrees"] = decInDegrees

In [128]:
new =[]
newRA =[]
for i in range(len(decInDegrees)):
  if decInDegrees[i] < 0:
    new.append(abs(decInDegrees[i]))
    if raInDegrees[i] <= 180:
      newRA.append(raInDegrees[i] + 180)
    else:
      newRA.append(raInDegrees[i] - 180)
  else:
    new.append(decInDegrees[i])
    newRA.append(raInDegrees[i])

data["NewDecDegrees"] = new
data["NewRA"] = newRA

In [129]:
data.describe()

,v_h,Zmag,U,Ne,RADegrees,DecDegrees,NewDecDegrees,NewRA
count,39446.000000,19204.000000,19204.000000,19204.000000,39446.000000,39446.000000,39446.000000,39446.000000
mean,7184.791563,14.912602,0.551760,0.145438,174.704077,20.349858,20.555053,176.648001
std,10668.136485,0.762431,0.855416,0.445358,86.812111,20.524835,20.319331,86.236931
min,-571.000000,4.300000,0.000000,0.000000,0.007083,-15.929221,0.000008,0.007083
25%,4561.000000,14.700000,0.000000,0.000000,126.467396,5.050088,5.279369,129.366354
50%,6824.000000,15.200000,0.000000,0.000000,179.532917,12.820934,12.900208,181.293750
75%,9280.000000,15.400000,1.000000,0.000000,227.047812,31.099020,31.099020,228.136771
max,926681.000000,15.700000,4.000000,5.000000,359.975000,98.979229,98.979229,359.984167


In [130]:
fig = px.scatter_polar(data, r="NewDecDegrees", theta="NewRA", color = "v_h", color_continuous_scale=[(0.00, "red"),   (.03, "red"),
                                                     (.03, "blue"), (.07, "blue"),
                                                     (.07, "magenta"), (.12, "magenta"),
                                                     (.12, "cyan"),(.25,"cyan"),
                                                     (.25, "green"),  (1, "green")],range_color=[0,100000]  )
fig
        

![ZCAT 2000](https://drive.google.com/uc?export=view&id=1xnuRUcgVl7UxQ1y6N3GP8EvV9U5IQ7xF)

![Constellations](https://drive.google.com/uc?export=view&id=18KT8Ozqm2k8xSgXej-XoB-qYwu-YalHt)